<center><h1>Informarion Retrival Project</h1></center>
<center><h1>Build an Intelligent Information Retrival System</h1></center>

## Importation des librairies et des données textuelles

In [1]:
import nltk
import numpy as np
import pandas as pd
import re
import json

In [2]:
documents = open("../cisi/CISI.ALL").read()
queries = open("../cisi/CISI.QRY").read()
documents[:1000], queries[:1000]

(".I 1\n.T\n18 Editions of the Dewey Decimal Classifications\n.A\nComaromi, J.P.\n.W\n   The present study is a history of the DEWEY Decimal\nClassification.  The first edition of the DDC was published\nin 1876, the eighteenth edition in 1971, and future editions\nwill continue to appear as needed.  In spite of the DDC's\nlong and healthy life, however, its full story has never\nbeen told.  There have been biographies of Dewey\nthat briefly describe his system, but this is the first\nattempt to provide a detailed history of the work that\nmore than any other has spurred the growth of\nlibrarianship in this country and abroad.\n.X\n1\t5\t1\n92\t1\t1\n262\t1\t1\n556\t1\t1\n1004\t1\t1\n1024\t1\t1\n1024\t1\t1\n.I 2\n.T \nUse Made of Technical Libraries\n.A \nSlater, M.\n.W\nThis report is an analysis of 6300 acts of use\nin 104 technical libraries in the United Kingdom.\nLibrary use is only one aspect of the wider pattern of\ninformation use.  Information transfer in libraries is\nrestrict

## Indexiation des des documents

### Extraire les termes avec split
### Extraire les termes en utilisant la tokenization
### Supprimer les mots vides
### Normaliser les terme
### Pondérer les termes

In [3]:
# Fonction pour extraire les données des fichiers
def split_extraction(documents):
    content = documents.split(".I ") # Split sur le numéro de document
    del content[0] # Suppression du premier élément qui est vide

    parts = content 
    data = [] # Liste qui contiendra les données
    for part in parts: # Pour chaque document
        
        temp = part.split(".T\n") # Split sur le titre
        if len(temp) == 1 : temp = part.split(".T \n") # Si le titre prend un espace avant de retourner à la ligne
        if len(temp) == 1 : temp = part.split(".T  \n") # Si le titre prend deux espaces avant de retourner à la ligne
        
        del temp[0] # Suppression du premier élément qui est vide
        aut_temp = temp[0].split(".A\n") if len(temp[0].split(".A\n")) >= 2 else temp[0].split(".A \n") # Split sur l'auteur (si l'auteur prend un espace avant de retourner à la ligne)
        if len(aut_temp) > 2 : # Si le document a plus d'un auteur
            while len(aut_temp) != 2: # On supprime l'intégralité des auteurs 
                del aut_temp[1]
        title, temp2 = aut_temp # On récupère le titre et le reste du document
        
        summary = temp2.split(".W\n")[-1].split(".X")[0] # On récupère le résumé et on supprime les références
        summary = summary.split(".W")[-1] # On re applique un split afin de gérer un cas particulier
        data.append([' '.join(title.split()).lower(),' '.join(summary.split()).lower()]) # On ajoute le titre et le résumé dans la liste
    return data
data = split_extraction(documents)
pd.DataFrame(data, columns = ["Titre","Résumé"])

,Titre,Résumé
0,18 editions of the dewey decimal classifications,the present study is a history of the dewey de...
1,use made of technical libraries,this report is an analysis of 6300 acts of use...
2,two kinds of power an essay on bibliographic c...,the relationships between the organization and...
3,systems analysis of a university library; fina...,the establishment of nine new universities in ...
4,a library management game: a report on a resea...,although the use of games in professional educ...
...,...,...
1455,world dynamics,over the last several decades interest in econ...
1456,world trends in library education,one of the most significant aspects of the evo...
1457,legal restrictions on exploitation of the pate...,the patent laws confer on a patentee power to ...
1458,language and thought,this book considers the basic aspects of this ...


In [4]:
# Fonction qui permet de fusionner le titre et le résumé
def flatten(l):
    return [item for sublist in l for item in sublist]

In [12]:
# Fonction qui permet de créer le document inverse
def get_documents_inverse(data, stem):
    i = 1
    documents_inverse = [] # Liste qui contiendra les données du document inverse
    for d in data: # Pour chaque document
        text = ""
        text += ' '.join(d) + " " # On fusionne le titre et le résumé

        rgx = '(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%?|\w+(?:\-\w+)*' # Regex du prof pour récupérer les mots
        text_token = nltk.RegexpTokenizer(rgx).tokenize(text) # On récupère les mots

        mots_vides = nltk.corpus.stopwords.words('english') # On récupère les mots vides
        termes_sans_mots_vides = [terme for terme in text_token if terme.lower() not in mots_vides] # On supprime les mots vides

        # On initialise les stemmers
        ps = nltk.PorterStemmer() 
        ls = nltk.LancasterStemmer()
        # On applique le stemmer choisit par l'utilisateur
        if stem == "p" : stemmed = [ps.stem(tt) for tt in termes_sans_mots_vides]
        elif stem == "l" : stemmed = [ls.stem(tt) for tt in termes_sans_mots_vides]

        freq_word_doc = nltk.FreqDist(stemmed) # On récupère la fréquence des mots

        # On ajoute les données dans la liste
        filtered_word_freq = [[word, i, freq] 
                                for word, freq in freq_word_doc.items()]
        i += 1 # On incrémente le numéro de document
        documents_inverse.append(filtered_word_freq) # On ajoute les données dans la liste

    return pd.DataFrame(flatten(documents_inverse), columns = ["Word","Document","Frequence"])
document_inverse = get_documents_inverse(data, "l")
document_inverse

,Word,Document,Frequence
0,18,1,1
1,edit,1,4
2,dewey,1,3
3,decim,1,2
4,class,1,2
...,...,...,...
75471,100,1460,1
75472,150,1460,1
75473,thousand,1460,1
75474,new,1460,1


In [10]:
# Fonction de ponderation
def ponderation(document_inverse):
    max_freq = document_inverse.groupby('Document')['Frequence'].max().to_frame() # On récupère la fréquence maximale par document
    max_freq.reset_index(inplace=True) # On reset l'index
    max_freq.rename(columns = {'Frequence':'Max Frequence'}, inplace = True) # On renomme la colonne
    df = document_inverse.merge(max_freq, on="Document") # On merge les deux dataframes

    nb_doc = document_inverse.groupby('Word')['Document'].count().to_frame() # On récupère le nombre de document par mot
    nb_doc.reset_index(inplace=True) # On reset l'index
    nb_doc.rename(columns = {'Document':'Nombre document contenu'}, inplace = True) # On renomme la colonne
    df = df.merge(nb_doc, on="Word") # On merge les deux dataframes

    # On calcule le poid
    df["Poid"] = np.multiply(
        np.divide( # On divise la fréquence par la fréquence maximale
            df["Frequence"], 
            df["Max Frequence"]
            ), 
            np.log10( 
                np.divide( # On divise le nombre de document par le nombre de document contenant le mot
                    len(document_inverse['Document'].unique()),  
                    df["Nombre document contenu"]
                ) + 1
            )
        )

    fichier_descripteur = df[["Document", "Word", "Frequence", "Poid"]].sort_values(by=["Document"]) # On trie par document
    fichier_inverse = df[["Word", "Document", "Frequence", "Poid"]].sort_values(by=["Word"]) # On trie par mot
    
    fichier_descripteur["Document"] = fichier_descripteur["Document"].astype(str) # On convertit les données en string
    fichier_descripteur["Document"] = "D"+fichier_descripteur["Document"] # On ajoute un D devant le numéro de document
    fichier_inverse["Document"] = fichier_inverse["Document"].astype(str) # On convertit les données en string
    fichier_inverse["Document"] = "D"+fichier_inverse["Document"]  # On ajoute un D devant le numéro de document

    return fichier_descripteur, fichier_inverse # On retourne les deux dataframes

fichier_descripteur, fichier_inverse = ponderation(document_inverse)


In [11]:
len(fichier_descripteur['Word'].unique()) # On vérifie que le nombre de document est bien égal à 1000

6044

In [13]:
fichier_descripteur.to_csv("../fichier_descripteur_Lancaster.csv")
fichier_descripteur

,Document,Word,Frequence,Poid
0,D1,18,1,0.565909
2272,D1,biograph,1,0.640870
2276,D1,brief,1,0.361933
2330,D1,describ,1,0.178153
2681,D1,system,1,0.141720
...,...,...,...,...
3223,D1460,system,1,0.113376
31050,D1460,world,2,0.538223
34649,D1460,journ,1,0.210475
59034,D1460,integr,1,0.297699


In [8]:
# Retourner la liste des termes d'un document donné (avec fréquences et poids)
list_of_terms = fichier_descripteur.loc[fichier_descripteur["Document"] == "D315"]
list_of_terms = list_of_terms[["Word", "Frequence", "Poid"]]
list_of_terms.sort_values(by=["Word"])

,Word,Frequence,Poid
19406,abstract,2,0.322756
5470,analysi,3,0.374750
39133,applic,1,0.148386
28507,approach,2,0.303914
53100,automat,7,1.193125
61172,conclus,1,0.210242
26622,content,1,0.188888
37327,date,1,0.248707
3191,detail,1,0.169869
11743,discuss,1,0.116819


In [14]:
fichier_inverse.to_csv("../fichier_inverse_Lancaster.csv")
fichier_inverse

,Word,Document,Frequence,Poid
71676,0,D443,1,0.210977
74638,0.18,D1090,1,0.316465
73314,0.5,D720,1,0.791163
73204,0.7%,D691,1,0.316465
62469,000,D1091,1,0.178558
...,...,...,...,...
58149,zipf,D1173,1,0.354370
60582,zon,D62,2,0.575391
73463,zoolog,D755,1,0.632930
75138,zuckerm,D1335,2,0.954639


In [10]:
# Retourner la liste des documents contenant un terme donné (avec fréquence et poids)
list_of_terms = fichier_inverse.loc[fichier_inverse["Word"] == "ddc"]
list_of_terms[["Document", "Frequence", "Poid"]]

,Document,Frequence,Poid
1088,D1356,1,0.822289
1086,D517,1,0.411145
1085,D13,1,1.233434
1087,D527,2,1.233434
1084,D1,2,1.233434


## Indexation des requetes

In [11]:
# Fonction pour extraire les requetes
def extract_queries(queries):
    content = queries.split(".I ") # Split sur le numéro de la requete
    del content[0] # On supprime le premier élément qui est vide

    parts = content
    data = [] # Liste qui contiendra les données

    for part in parts: # On parcourt chaque partie

        if ".T" in part: # Si le titre est présent
            temp = part.split(".T\n") # On split sur le titre
            del temp[0] # On supprime le premier élément qui est vide
            aut_temp = temp[0].split(".A\n") # On split sur l'auteur
            title, temp2 = aut_temp # On récupère le titre et le reste
            
            query = temp2.split(".W\n")[1].split(".B\n")[0] # On récupère la requete et on supprime les references
            data.append([' '.join(title.split()).lower(),' '.join(query.split()).lower()]) # On ajoute les données dans la liste
        else:
            query = part.split(".W\n")[1] # On récupère la requete
            data.append(['',' '.join(query.split()).lower()]) # On ajoute les données dans la liste
    return data # On retourne les données

data_queries = extract_queries(queries)
pd.DataFrame(data_queries, columns = ["Titre","Query"])

,Titre,Query
0,,what problems and concerns are there in making...
1,,"how can actually pertinent data, as opposed to..."
2,,what is information science? give definitions ...
3,,image recognition and any other methods of aut...
4,,what special training will ordinary researcher...
...,...,...
107,a program for machine-mediated searching,a technique of online instruction and assistan...
108,author cocitation: a literature measure of int...,it is shown that the mapping of a particular a...
109,progress in documentation. word processing: an...,"the ""office of the future,"" ""office technology..."
110,document clustering using an inverted file app...,an automated document clustering procedure is ...


In [12]:
# Fonction qui transforme les requetes au format approprié pour etre utilisé par les SRI
def format_queries(data, stem):
    i = 1
    info_queries = [] # Liste qui contiendra les données
    for d in data: # On parcourt chaque requete
        text = ""
        if len(d[0]) != 0: text += ' '.join(d) + " " # Si le titre est présent on le fusionne avec la requete
        else : text += ''.join(d) + " " # Sinon on ne prend que la requete

        rgx = '(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%? |\w+(?:\-\w+)*' # Regex du prof pour les mots
        text_token = nltk.RegexpTokenizer(rgx).tokenize(text) # On tokenize la requete

        mots_vides = nltk.corpus.stopwords.words('english') # On récupère les mots vides
        termes_sans_mots_vides = [terme for terme in text_token if terme.lower() not in mots_vides] # On supprime les mots vides

        # On initialise les stemmers
        ps = nltk.PorterStemmer()
        ls = nltk.LancasterStemmer()
        # On selectionne le stemmer choisi par l'utilisateur et on stemme les termes
        if stem == "p": stemmed = [ps.stem(tt) for tt in termes_sans_mots_vides]
        elif stem == "l": stemmed = [ls.stem(tt) for tt in termes_sans_mots_vides]

        filtered_word_freq = [["Q"+str(i), word] for word in stemmed if not word.isdigit()] # On calcule la fréquence des mots
        i += 1
        info_queries.append(filtered_word_freq) # On ajoute les données dans la liste

    return pd.DataFrame(flatten(info_queries), columns = ["Query","Word"])
info_queries = format_queries(data_queries, "p")
info_queries.to_csv("../info_queries.csv")
info_queries

,Query,Word
0,Q1,problem
1,Q1,concern
2,Q1,make
3,Q1,descript
4,Q1,titl
...,...,...
5188,Q112,algorithm
5189,Q112,compar
5190,Q112,previous
5191,Q112,describ


## Indexation des requetes booléenne

In [13]:
# Fonction qui transforme les requetes au format approprié pour etre utilisé par le SRI booléen
def format_bool_queries():
    bool_queries = open("../cisi/CISI.BLN").read().replace("\n","") # On lit le fichier et on supprime les sauts de ligne
    bool_queries = bool_queries.replace("\t","") # On supprime les tabulations
    bool_queries = bool_queries.split(";") # On split sur le point virgule

    del bool_queries[0] # On supprime le premier élément qui est vide

    boolean_queries = [] # Liste qui contiendra les données

    for bq in bool_queries: # On parcourt chaque requete
        if "= " in bq: 
            code, bool_query = bq.split("= ") # On split sur le code et la requete
            boolean_queries.append([code[1:].upper(), bool_query]) # On ajoute les données dans la liste
    return pd.DataFrame(boolean_queries,columns =["Code","Bool Query"]) # On retourne les données
bool_queriers = format_bool_queries()
bool_queriers.to_csv("../bool_queries.csv")
bool_queriers

,Code,Bool Query
0,Q1,"#and ('titles', #or ('automatically', 'retriev..."
1,Q2,"#and (#or('data', 'information'), #..."
2,Q3,"#and ('information',#or ('science', 'definitio..."
3,Q4,"#or (#and ('image', 'recognition'), ..."
4,Q5,"#and (#or ('training', 'use', 'need', 'systems..."
5,Q6,"#and (#or ('communication', 'verbal'), #or (..."
6,Q7,"#and ( #or ('systems', 'data-processing', ..."
7,Q8,"#and (#or ('languages', 'indexing'), #or ('..."
8,Q9,"#and (#or ('information', 'retrieval', 'system..."
9,Q10,"#or (#and ('group', 'mathematics'), ..."


## Création du dataset pour le SRI basé sur le text mining

### Dataset pour DBSCAN

In [14]:
unique_terms = list(fichier_descripteur['Word'].unique()) # On récupère les termes uniques
df = pd.DataFrame(columns = unique_terms) # On crée un dataframe avec les termes uniques comme colonnes
for document in fichier_descripteur['Document'].unique(): # On parcourt chaque document
    temp = fichier_descripteur.loc[fichier_descripteur['Document'] == document] # On récupère les lignes du document
    temp = temp.drop(['Frequence'], axis=1) # On supprime la colonne de la fréquence
    df.loc[document] = pd.Series((pd.Series(temp.Poid.values,index=temp.Word.values).to_dict())) # On ajoute les données dans le dataframe
df.fillna(0, inplace=True) # On remplace les valeurs manquantes par 0
df.to_csv("../dataset_dbscan.csv") # On sauvegarde le dataframe
df

,18,told,biographi,briefli,describ,system,attempt,provid,decim,detail,...,anglo,supposit,morn,claus,inventor,sell,patente,monopoli,15%,9%
D1,0.565909,0.64087,0.672031,0.410718,0.201332,0.145940,0.275780,0.209136,0.982483,0.29727,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D2,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D3,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D4,0.000000,0.00000,0.000000,0.000000,0.000000,0.116752,0.000000,0.167309,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D5,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D1456,0.000000,0.00000,0.000000,0.000000,0.000000,0.233504,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D1457,0.000000,0.00000,0.000000,0.000000,0.000000,0.097293,0.183853,0.000000,0.000000,0.00000,...,0.527442,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
D1458,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,1.054883,1.054883,1.054883,1.054883,1.054883,1.054883,1.054883,0.000000,0.000000
D1459,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


## Création du Dataframe d'évaluation

In [23]:
eval_content = open("../cisi/CISI.REL").read()
rows = eval_content.split("\n")
eval_df = []
for row in rows:
    temp_vals = row.split("\t")[0].strip()
    vals = temp_vals.split(" ")
    while "" in vals:
        vals.remove("")
    if len(vals) == 2: eval_df.append(["Q"+vals[0], "D"+vals[1]])
eval_df = pd.DataFrame(eval_df, columns = ["Query","Document"])
eval_df.to_csv("../evaluation.csv")
eval_df

,Query,Document
0,Q1,D28
1,Q1,D35
2,Q1,D38
3,Q1,D42
4,Q1,D43
...,...,...
3109,Q111,D422
3110,Q111,D448
3111,Q111,D485
3112,Q111,D503
